# Семинар 2

Сегодня посмотрим как можно решать задачу text-style transfer, а главное посмотрим как можно подойти к методике валидации качества модели. В качестве задачи мы попробуем обучить модель детоксификации данных.

## Посмотроим на данные

In [14]:
import os
import json
import sys
import gc
from typing import Tuple
from pathlib import Path 

from tqdm.auto import tqdm, trange
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property

In [15]:
data_path = './data/input/train.tsv'

In [16]:
df = pd.read_csv(data_path, sep='\t')
df = df.fillna('')

In [17]:
df.sample(5)

,index,toxic_comment,neutral_comment1,neutral_comment2,neutral_comment3
6708,6708,Блин ебаный пиздец хотел с тобой увидится но н...,хотел с тобой увидится но не получилось:( изо ...,,
979,979,да надо было е му как широков заебашить прям н...,да надо было ему как широков забить прям на поле,,
5739,5739,"перезагорал, ебало все облезло( и все(","Перезагорал, лицо всё облезло( и все(",,
147,147,ты мразь почему стыдишься русского языка ??,почему вы стыдиться русского языка?,,
1340,1340,шакалы отстаньте от золотого человека уроды ва...,Отстаньте от золотого человека. Вам до него ох...,"Отстаньте от золотого человека, вам далеко до ...",


In [18]:
df.shape

(6948, 5)

## Обучим простую модель T5

In [19]:
df_train_toxic = []
df_train_neutral = []

for index, row in df.iterrows():
    references = row[['neutral_comment1', 'neutral_comment2', 'neutral_comment3']].tolist()
    
    for reference in references:
        if len(reference) > 0:
            df_train_toxic.append(row['toxic_comment'])
            df_train_neutral.append(reference)
        else:
            break

In [20]:
df = pd.DataFrame({
    'toxic_comment': df_train_toxic,
    'neutral_comment': df_train_neutral
})

df = shuffle(df)

Подготовим данные для обучения

In [21]:
class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx < len(self.x['input_ids'])
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item
    
    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n # * 2

In [23]:
from typing import List, Dict, Union

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        ) 
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']
        
        return {k: torch.tensor(v) for k, v in batch.items()}

In [24]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [25]:
def evaluate_model(model, test_dataloader):
    num = 0
    den = 0

    for batch in test_dataloader:
        with torch.no_grad():
            loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
            num += len(batch) * loss.item()
            den += len(batch)
    val_loss = num / den
    return val_loss

In [26]:
def train_loop(
    model, train_dataloader, val_dataloader, 
    max_epochs=30, 
    max_steps=1_000, 
    lr=3e-5,
    gradient_accumulation_steps=1, 
    cleanup_step=100,
    report_step=300,
    window=100,
):
    cleanup()
    optimizer = torch.optim.Adam(params = [p for p in model.parameters() if p.requires_grad], lr=lr)

    ewm_loss = 0
    step = 0
    model.train()

    for epoch in trange(max_epochs):
        print(step, max_steps)
        if step >= max_steps:
            break
        tq = tqdm(train_dataloader)
        for i, batch in enumerate(tq):
            try:
                batch['labels'][batch['labels']==0] = -100
                loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
                loss.backward()
            except Exception as e:
                print('error on step', i, e)
                loss = None
                cleanup()
                continue
            if i and i % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                step += 1
                if step >= max_steps:
                    break

            if i % cleanup_step == 0:
                cleanup()

            w = 1 / min(i+1, window)
            ewm_loss = ewm_loss * (1-w) + loss.item() * w
            tq.set_description(f'loss: {ewm_loss:4.4f}')

            if (i and i % report_step == 0 or i == len(train_dataloader)-1)  and val_dataloader is not None:
                model.eval()
                eval_loss = evaluate_model(model, val_dataloader)
                model.train()
                print(f'epoch {epoch}, step {i}/{step}: train loss: {ewm_loss:4.4f}  val loss: {eval_loss:4.4f}')
                
            if step % 1000 == 0:
                model.save_pretrained(f't5_base_{dname}_{steps}')
        
    cleanup()

In [27]:
def train_model(x, y, model_name, test_size=0.1, batch_size=32, **kwargs):
    model = T5ForConditionalGeneration.from_pretrained(model_name).cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    x1, x2, y1, y2 = train_test_split(x, y, test_size=test_size, random_state=42)
    train_dataset = PairsDataset(tokenizer(x1), tokenizer(y1))
    test_dataset = PairsDataset(tokenizer(x2), tokenizer(y2))
    
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False, shuffle=True, collate_fn=data_collator)
    val_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=False, shuffle=True, collate_fn=data_collator)

    train_loop(model, train_dataloader, val_dataloader, **kwargs)
    return model

In [28]:
model_name = 'sberbank-ai/ruT5-base'

In [29]:
cleanup()

In [30]:
datasets = {
    'train': df
}

Попробуем перебирать разное количество шагов обучения для того чтобы понять, когда стоит остановить/продолжить обучение

In [33]:
for steps in [300, 1000, 3000, 10000]:
    for dname, d in datasets.items():
        print(f'\n\n\n  {dname}  {steps} \n=====================\n\n')
        model = train_model(d['toxic_comment'].tolist(), d['neutral_comment'].tolist(), model_name=model_name, batch_size=16, max_epochs=1000, max_steps=steps)
        model.save_pretrained(f't5_base_{dname}_{steps}')




  train  300 




  0%|          | 0/1000 [00:00<?, ?it/s]

0 300


  0%|          | 0/624 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

  0%|          | 0/1000 [00:00<?, ?it/s]

0 10000


  0%|          | 0/624 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

Теперь посмотрим что у нас на inference модели

In [36]:
df = pd.read_csv('./data/input/dev.tsv', sep='\t')
toxic_inputs = df['toxic_comment'].tolist()

Подгрузим один из обученных нами чекпоинтов

In [39]:
base_model_name = 'sberbank-ai/ruT5-base'
model_name = './t5_base_train_3000'

In [40]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).cuda()

In [41]:
def paraphrase(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs, 
        num_return_sequences=n or 1, 
        do_sample=False, 
        temperature=temperature, 
        repetition_penalty=3.0, 
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [42]:
print(paraphrase(['Дмитрий вы ебанулись, уже все выложено'], model, temperature=50.0, beams=10))

['Дмитрий, уже все выложено']


/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `50.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [43]:
para_results = []
problematic_batch = [] #if something goes wrong you can track such bathces
batch_size = 8

for i in tqdm(range(0, len(toxic_inputs), batch_size)):
    batch = [sentence for sentence in toxic_inputs[i:i + batch_size]]
    try:
        para_results.extend(paraphrase(batch, model, temperature=0.0))
    except Exception as e:
        print(i)
        para_results.append(toxic_inputs[i:i + batch_size])

  0%|          | 0/100 [00:00<?, ?it/s]

/home/snv/Projects/courses/SF_generation_NLP/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [44]:
with open('./data/output/t5_10000_steps_dev.txt', 'w') as file:
    file.writelines([sentence+'\n' for sentence in para_results])

Далее мы с вами посмотрим, а как же можно все таки в автоматизированном режиме оценивать качество получившейся модели

### А как оценивать успешность?

Сохраним в отдельном файле все полезные методы, которые мы будем использовать для автоматической проверки и валидации 

In [33]:
from detoxification_metrics import evaluate_style

In [9]:
with open('./data/output/t5_base_10000_dev.txt', 'r') as file:
    preds = file.readlines()
preds = [sentence.strip() for sentence in preds]
len(preds)

800

Воспользуемся открытой моделью-классификатором токсичных текстов, и попробуем ее в качестве первого этапа валидации

In [ ]:
def load_model(model_name=None, model=None, tokenizer=None,
               model_class=AutoModelForSequenceClassification, use_cuda=True):
    if model is None:
        if model_name is None:
            raise ValueError('Either model or model_name should be provided')
        model = model_class.from_pretrained(model_name)
        if torch.cuda.is_available() and use_cuda:
            model.cuda()
    if tokenizer is None:
        if model_name is None:
            raise ValueError('Either tokenizer or model_name should be provided')
        tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [22]:
style_model, style_tokenizer = load_model('SkolkovoInstitute/russian_toxicity_classifier', use_cuda=True)

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [23]:
accuracy = evaluate_style(
    model = style_model,
    tokenizer = style_tokenizer,
    texts = preds,
    target_label=0,  # 1 is toxic, 0 is neutral
    batch_size=32, 
    verbose=True
)

  0%|          | 0/25 [00:00<?, ?it/s]

In [24]:
print(f'Style transfer accuracy (STA):  {np.mean(accuracy)}')

Style transfer accuracy (STA):  0.753741443157196


### метод 2

In [25]:
from ru_detoxification_metrics import evaluate_cosine_similarity

Энкодер предложений (sentence encoder) – это модель, которая получает на вход текст предложения, а на выходе отдаёт многомерный вектор (например, 768-мерный), примерно описывающий смысл этого предложения. То есть такой, что у предложений, похожих друг на друга по смыслу, векторы похожи друг на друга геометрически. Как мы увидели на прошлом занятии, энкодеры предложений можно использовать для классификации текстов и массы других полезных задач.

LaBSE (language-agnostic BERT sentence embeddings) – это модель, предложенная в статье 2020 года от исследователей из Google. По архитектуре это BERT, а обучался он на выборке текстов на 100+ языков в многозадачном режиме. Основная задача – сближать друг с другом эмбеддинги предложений с одинаковым смыслом на разных языках, и с этой задачей модель справляется очень хорошо. Благодаря этой способности можно, например, обучать модель классифицировать английские тексты, а потом применять на русских, или находить в большом корпусе пары предложений на разных языках, являющиеся переводами друг друга.

Именно поэтому мы воспользуемся данной моделью и посмотрим неа косинусную близость получившихся векторов исходных тектсов и трансформированных моделью Т5.

In [26]:
meaning_model, meaning_tokenizer = load_model('cointegrated/LaBSE-en-ru', use_cuda=True, model_class=AutoModel)

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/516M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [34]:
similarity = evaluate_cosine_similarity(
    model = meaning_model,
    tokenizer = meaning_tokenizer,
    original_texts = toxic_inputs,
    rewritten_texts = preds,
    batch_size=32,
    verbose=True,
    )

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [35]:
print(f'Meaning preservation (SIM):  {np.mean(similarity)}')

Meaning preservation (SIM):  0.8054955005645752


### метод 3

In [36]:
from ru_detoxification_metrics import evaluate_cola_relative

Модель, которую возьмем для этой задачи создана для оценки естественности коротких текстов на русском языке. Она была обучена отличать тексты, написанные человеком, от их искаженных версий.

Источники искажений: случайная замена, удаление, добавление, перетасовка и изменение интонации слов и символов, случайные изменения заглавных букв, обратный перевод, заполнение случайных пробелов с помощью моделей T5 и RoBERTA. Для каждого исходного текста отбирались три поврежденных текста, так что модель равномерно смещена в сторону неестественной метки.

In [38]:
cola_model, cola_tolenizer = load_model('SkolkovoInstitute/rubert-base-corruption-detector', use_cuda=True)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [39]:
fluency = evaluate_cola_relative(
    model = cola_model,
    tokenizer = cola_tolenizer,
    original_texts = toxic_inputs,
    rewritten_texts = preds,
    target_label=1,
    batch_size=32,
    verbose=True
)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
print(f'Fluency score (FL):  {np.mean(fluency)}')

### Обобщенная оценка

Как мы уже разбирали на лекции, зачастую ваша финальная метрика не является каким то одной фиксированной метрикой, или представляет из себя агрегацию/ансамблирование/иерархию сразу нескольких производных метрик.

Для этой задачи придумаем похожую аккумулированную оценку и воспользуемся ей как финальной метрикой качества модели

In [40]:
joint = accuracy * similarity * fluency

In [41]:
print(f'Joint score (J):   {np.mean(joint)}')

Joint score (J):   0.504615306854248


In [42]:
from nltk.translate.chrf_score import corpus_chrf

In [43]:
corpus_chrf(neutral_references, preds)

0.5790396890355219